<div style="background-color:#eef5ff; padding:12px; border-radius:4px;">

# DATA 557 Final Project  
## Promotion from Associate to Full Professor  

</div>

<div style="background-color:#eef5ff; padding:12px; border-radius:4px;">

### Objective

Determine whether sex-based differences exist in the probability of promotion  
from Associate Professor to Full Professor.

We begin by constructing a faculty-level dataset and defining a binary  
promotion outcome (ever promoted vs never promoted).

</div>

## Setup

#### Imports

In [1]:
import numpy as np
import pandas as pd

#### Load Data

In [8]:
DATA_FILE_PATH = "Data557_FinalProject_Dataset.txt"

salary_df = pd.read_csv(DATA_FILE_PATH,
                        sep=r"\s+",
                        na_values="NA")
salary_df

,case,id,sex,deg,yrdeg,field,startyr,year,rank,admin,salary
0,1,1,F,Other,92,Other,95,95,Assist,0,6684.0
1,2,2,M,Other,91,Other,94,94,Assist,0,4743.0
2,3,2,M,Other,91,Other,94,95,Assist,0,4881.0
3,4,4,M,PhD,96,Other,95,95,Assist,0,4231.0
4,5,6,M,PhD,66,Other,91,91,Full,1,11182.0
...,...,...,...,...,...,...,...,...,...,...,...
19787,19788,1770,M,Other,51,Other,64,91,Full,0,5318.0
19788,19789,1770,M,Other,51,Other,64,92,Full,0,5472.0
19789,19790,1770,M,Other,51,Other,64,93,Full,0,5551.0
19790,19791,1770,M,Other,51,Other,64,94,Full,0,5551.0


#### Data Cleaning

In [9]:
# Clean column names just in case there is whitespace
salary_df.columns = salary_df.columns.str.strip()


Dataset shape: (19792, 11)
Columns: ['case', 'id', 'sex', 'deg', 'yrdeg', 'field', 'startyr', 'year', 'rank', 'admin', 'salary']


,case,id,sex,deg,yrdeg,field,startyr,year,rank,admin,salary
0,1,1,F,Other,92,Other,95,95,Assist,0,6684.0
1,2,2,M,Other,91,Other,94,94,Assist,0,4743.0
2,3,2,M,Other,91,Other,94,95,Assist,0,4881.0
3,4,4,M,PhD,96,Other,95,95,Assist,0,4231.0
4,5,6,M,PhD,66,Other,91,91,Full,1,11182.0


#### Check for NA values

In [10]:
print(salary_df.isna().sum())
print(salary_df["rank"].value_counts(dropna=False))
print(salary_df["sex"].value_counts(dropna=False))

case       0
id         0
sex        0
deg        0
yrdeg      0
field      0
startyr    0
year       0
rank       4
admin      0
salary     0
dtype: int64
rank
Full      9211
Assoc     6529
Assist    4048
NaN          4
Name: count, dtype: int64
sex
M    15866
F     3926
Name: count, dtype: int64


#### Drop rows with missing rank

In [11]:
rows_before = salary_df.shape[0]

salary_clean_df = salary_df.dropna(subset=["rank"]).copy()

rows_after = salary_clean_df.shape[0]

print("Rows before:", rows_before)
print("Rows after dropping missing rank:", rows_after)
print("Rows dropped:", rows_before - rows_after)

salary_clean_df["rank"].value_counts()

Rows before: 19792
Rows after dropping missing rank: 19788
Rows dropped: 4


rank
Full      9211
Assoc     6529
Assist    4048
Name: count, dtype: int64

### Data Exploration: Longitudinal 

#### Check if Data are longitudinal- if so id values should appear multiple times

In [13]:
rows_per_faculty = (salary_clean_df
                    .groupby("id")
                    .size())

print("Summary of number of rows per faculty:")
print(rows_per_faculty.describe())

Summary of number of rows per faculty:
count    1597.000000
mean       12.390733
std         6.718013
min         1.000000
25%         6.000000
50%        13.000000
75%        20.000000
max        20.000000
dtype: float64


#### Check how many faculty appear only once

In [14]:
single_year_faculty = (rows_per_faculty == 1).sum()
multi_year_faculty = (rows_per_faculty > 1).sum()

print("Faculty with 1 year:", single_year_faculty)
print("Faculty with multiple years:", multi_year_faculty)

Faculty with 1 year: 72
Faculty with multiple years: 1525


#### Check that the year varies within a faculty member

In [15]:
years_per_faculty = (salary_clean_df
                     .groupby("id")["year"]
                     .nunique())

print(years_per_faculty.describe())

count    1597.000000
mean       12.390733
std         6.718013
min         1.000000
25%         6.000000
50%        13.000000
75%        20.000000
max        20.000000
Name: year, dtype: float64


### Subset Data

#### Identify faculty who ever become associate

In [12]:
associate_ids = (salary_clean_df
                 .loc[salary_clean_df["rank"] == "Assoc", "id"]
                 .unique())

associate_df = salary_clean_df[
    salary_clean_df["id"].isin(associate_ids)].copy()

print("Number of faculty who ever become Associate:", len(associate_ids))
print("Subset shape (all years for those faculty):", associate_df.shape)

Number of faculty who ever become Associate: 984
Subset shape (all years for those faculty): (14191, 11)


#### First Associate Year per Faculty

In [17]:
first_associate_year_df = (associate_df[associate_df["rank"] == "Assoc"]
                           .groupby("id", as_index=False)["year"]
                           .min()
                           .rename(columns={"year": "first_associate_year"}))

first_associate_year_df

,id,first_associate_year
0,7,79
1,9,90
2,10,80
3,13,76
4,14,92
...,...,...
979,1764,84
980,1765,76
981,1766,76
982,1768,90


#### Identify Faculty Ever Promoted After Associate

In [18]:
associate_df = associate_df.merge(first_associate_year_df,
                                  on="id",
                                  how="left")


promoted_ids = (associate_df[
                (associate_df["rank"] == "Full") &
                (associate_df["year"] > associate_df["first_associate_year"])
                ]["id"].unique())

print("Number ever promoted after Associate:", len(promoted_ids))

Number ever promoted after Associate: 547


#### Construct Faculty-Level Dataset

In [19]:
faculty_level_df = (associate_df
                    .sort_values(["id", "year"])
                    .groupby("id", as_index=False)
                    .first()[["id", "sex", "deg", "yrdeg", "field", "startyr"]])

faculty_level_df = faculty_level_df.merge(first_associate_year_df,
                                          on="id",
                                          how="left")

faculty_level_df["ever_promoted"] = (faculty_level_df["id"].isin(promoted_ids).astype(int))

print("Faculty-level dataset shape:", faculty_level_df.shape)
faculty_level_df.head()

Faculty-level dataset shape: (984, 8)


,id,sex,deg,yrdeg,field,startyr,first_associate_year,ever_promoted
0,7,M,PhD,70,Other,71,79,0
1,9,M,PhD,82,Other,87,90,0
2,10,M,PhD,68,Arts,80,80,1
3,13,M,PhD,68,Prof,69,76,1
4,14,M,PhD,79,Other,92,92,1


In [20]:
faculty_level_df.describe()

,id,yrdeg,startyr,first_associate_year,ever_promoted
count,984.000000,984.000000,984.000000,984.000000,984.000000
mean,871.450203,75.476626,79.413618,84.300813,0.555894
std,501.548911,7.080087,7.956690,6.374300,0.497119
min,7.000000,51.000000,50.000000,76.000000,0.000000
25%,452.250000,70.000000,73.000000,78.000000,0.000000
50%,855.500000,75.000000,80.000000,84.000000,1.000000
75%,1303.250000,81.000000,86.000000,90.000000,1.000000
max,1770.000000,92.000000,95.000000,95.000000,1.000000


### Key Findings: 

- 984 faculty who ever became Associate
- 547 of them were eventually promoted to Full
- Overall Promotion rate ≈ 55.6%    (547 / 984 ≈ 0.556)

#### Note about Promotion Followup Time & "Right tail bias":
- Someone who became Associate in 1995 has zero follow-up time.
- Someone who became Associate in 1994 has at most one year of follow-up.
- Someone who became Associate in 1976 has nearly 20 years of follow-up.

#### Look at dist of first associate year by sex

In [21]:
faculty_level_df.groupby("sex")["first_associate_year"].describe()

,count,mean,std,min,25%,50%,75%,max
sex,,,,,,,,
F,238.0,86.764706,5.828355,76.0,82.0,88.0,92.0,95.0
M,746.0,83.514745,6.344086,76.0,76.0,83.0,89.0,95.0


In [22]:
pd.crosstab(faculty_level_df["sex"],
            pd.cut(
                faculty_level_df["first_associate_year"],
                bins=[75, 80, 85, 90, 95],
                right=True))

first_associate_year,"(75, 80]","(80, 85]","(85, 90]","(90, 95]"
sex,,,,
F,48,49,58,83
M,304,139,163,140


In [23]:
cohort_promotion = (
    faculty_level_df
    .groupby(["sex", "first_associate_year"])["ever_promoted"]
    .mean()
    .reset_index()
)

cohort_promotion.head()

,sex,first_associate_year,ever_promoted
0,F,76,0.875000
1,F,77,0.800000
2,F,78,1.000000
3,F,79,0.545455
4,F,80,0.833333


In [24]:
faculty_level_df["associate_cohort_bin"] = pd.cut(
    faculty_level_df["first_associate_year"],
    bins=[75, 80, 85, 90, 95],
    right=True
)

promotion_by_cohort = (
    faculty_level_df
    .groupby(["sex", "associate_cohort_bin"])["ever_promoted"]
    .mean()
    .reset_index()
)

promotion_by_cohort

C:\Users\clark.roll\AppData\Local\Temp\ipykernel_8320\1805232603.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["sex", "associate_cohort_bin"])["ever_promoted"]


,sex,associate_cohort_bin,ever_promoted
0,F,"(75, 80]",0.791667
1,F,"(80, 85]",0.653061
2,F,"(85, 90]",0.500000
3,F,"(90, 95]",0.012048
4,M,"(75, 80]",0.779605
5,M,"(80, 85]",0.769784
6,M,"(85, 90]",0.564417
7,M,"(90, 95]",0.078571


In [26]:
faculty_level_df["first_associate_year"] = faculty_level_df["first_associate_year"].astype(int)

In [27]:
faculty_level_df["assoc_year_centered"] = (
    faculty_level_df["first_associate_year"]
    - faculty_level_df["first_associate_year"].mean()
)

# Creating CSV For Salary Comparison

In [28]:
# Identify the Year of Promotion
# First Full year after Associate for promoted faculty
promotion_year_df = (
    associate_df[
        (associate_df["rank"] == "Full") &
        (associate_df["year"] > associate_df["first_associate_year"])
    ]
    .groupby("id", as_index=False)["year"]
    .min()
    .rename(columns={"year": "promotion_year"})
)

promotion_year_df.head()

,id,promotion_year
0,10,93
1,13,82
2,14,94
3,16,89
4,17,83


In [29]:
# Create Promoted Faculty Dataset
promoted_faculty_df = faculty_level_df[
    faculty_level_df["ever_promoted"] == 1
].copy()

promoted_faculty_df = promoted_faculty_df.merge(
    promotion_year_df,
    on="id",
    how="left"
)

print("Promoted faculty dataset shape:", promoted_faculty_df.shape)
promoted_faculty_df.head()

Promoted faculty dataset shape: (547, 11)


,id,sex,deg,yrdeg,field,startyr,first_associate_year,ever_promoted,associate_cohort_bin,assoc_year_centered,promotion_year
0,10,M,PhD,68,Arts,80,80,1,"(75, 80]",-4.300813,93
1,13,M,PhD,68,Prof,69,76,1,"(75, 80]",-8.300813,82
2,14,M,PhD,79,Other,92,92,1,"(90, 95]",7.699187,94
3,16,M,PhD,73,Other,79,82,1,"(80, 85]",-2.300813,89
4,17,M,PhD,72,Other,72,77,1,"(75, 80]",-7.300813,83


In [31]:
# Add salary at promotion: 
salary_at_promotion_df = (
    salary_clean_df
    .merge(promotion_year_df, on="id", how="inner")
)

salary_at_promotion_df = salary_at_promotion_df[
    salary_at_promotion_df["year"] == salary_at_promotion_df["promotion_year"]
][["id", "salary"]]

salary_at_promotion_df = salary_at_promotion_df.rename(
    columns={"salary": "salary_at_promotion"}
)

promoted_faculty_df = promoted_faculty_df.merge(
    salary_at_promotion_df,
    on="id",
    how="left"
)

promoted_faculty_df.head()

,id,sex,deg,yrdeg,field,startyr,first_associate_year,ever_promoted,associate_cohort_bin,assoc_year_centered,promotion_year,salary_at_promotion
0,10,M,PhD,68,Arts,80,80,1,"(75, 80]",-4.300813,93,7180.0
1,13,M,PhD,68,Prof,69,76,1,"(75, 80]",-8.300813,82,3322.0
2,14,M,PhD,79,Other,92,92,1,"(90, 95]",7.699187,94,9025.0
3,16,M,PhD,73,Other,79,82,1,"(80, 85]",-2.300813,89,5290.0
4,17,M,PhD,72,Other,72,77,1,"(75, 80]",-7.300813,83,2994.0


In [32]:
OUTPUT_FILE = "promoted_faculty_dataset.csv"

promoted_faculty_df.to_csv(OUTPUT_FILE, index=False)

print("CSV exported:", OUTPUT_FILE)

CSV exported: promoted_faculty_dataset.csv


In [33]:
# ==============================
# Promotion metadata
# ==============================

# Promoted faculty ids
promoted_ids = faculty_level_df.loc[faculty_level_df["ever_promoted"] == 1, "id"].unique()

print("Number of promoted faculty:", len(promoted_ids))

# First Full year after Associate for promoted faculty
promotion_year_df = (
    associate_df[
        (associate_df["rank"] == "Full") &
        (associate_df["year"] > associate_df["first_associate_year"])
    ]
    .groupby("id", as_index=False)["year"]
    .min()
    .rename(columns={"year": "promotion_year"})
)

print("Promotion-year rows:", promotion_year_df.shape[0])
promotion_year_df.head()

Number of promoted faculty: 547
Promotion-year rows: 547


,id,promotion_year
0,10,93
1,13,82
2,14,94
3,16,89
4,17,83


In [34]:
# ==============================
# Dataset 1: promoted_faculty_summary
# One row per promoted faculty
# ==============================

promoted_faculty_summary_df = (
    faculty_level_df
    .loc[faculty_level_df["ever_promoted"] == 1]
    .merge(promotion_year_df, on="id", how="left")
)

print("promoted_faculty_summary_df shape:", promoted_faculty_summary_df.shape)
promoted_faculty_summary_df.head()

promoted_faculty_summary_df shape: (547, 11)


,id,sex,deg,yrdeg,field,startyr,first_associate_year,ever_promoted,associate_cohort_bin,assoc_year_centered,promotion_year
0,10,M,PhD,68,Arts,80,80,1,"(75, 80]",-4.300813,93
1,13,M,PhD,68,Prof,69,76,1,"(75, 80]",-8.300813,82
2,14,M,PhD,79,Other,92,92,1,"(90, 95]",7.699187,94
3,16,M,PhD,73,Other,79,82,1,"(80, 85]",-2.300813,89
4,17,M,PhD,72,Other,72,77,1,"(75, 80]",-7.300813,83


In [35]:
OUTPUT_FILE_1 = "promoted_faculty_summary.csv"
promoted_faculty_summary_df.to_csv(OUTPUT_FILE_1, index=False)
print("Wrote:", OUTPUT_FILE_1)

Wrote: promoted_faculty_summary.csv


In [36]:
# ==============================
# Dataset 2: promoted_faculty_longitudinal
# All faculty-year rows for promoted faculty
# ==============================

promoted_faculty_longitudinal_df = (
    salary_clean_df[salary_clean_df["id"].isin(promoted_ids)]
    .merge(
        faculty_level_df[["id", "first_associate_year"]],
        on="id",
        how="left"
    )
    .merge(
        promotion_year_df,
        on="id",
        how="left"
    )
    .sort_values(["id", "year"])
)

print("promoted_faculty_longitudinal_df shape:", promoted_faculty_longitudinal_df.shape)
promoted_faculty_longitudinal_df.head()

promoted_faculty_longitudinal_df shape: (9127, 13)


,case,id,sex,deg,yrdeg,field,startyr,year,rank,admin,salary,first_associate_year,promotion_year
0,40,10,M,PhD,68,Arts,80,80,Assoc,0,3250.0,80,93
1,41,10,M,PhD,68,Arts,80,81,Assoc,0,3671.0,80,93
2,42,10,M,PhD,68,Arts,80,82,Assoc,0,3831.0,80,93
3,43,10,M,PhD,68,Arts,80,83,Assoc,0,4042.0,80,93
4,44,10,M,PhD,68,Arts,80,84,Assoc,0,4317.0,80,93


In [37]:
OUTPUT_FILE_2 = "promoted_faculty_longitudinal.csv"
promoted_faculty_longitudinal_df.to_csv(OUTPUT_FILE_2, index=False)
print("Wrote:", OUTPUT_FILE_2)

Wrote: promoted_faculty_longitudinal.csv


In [38]:
# ==============================
# Dataset 3: promotion_event_dataset
# One row per promoted faculty with salary change at promotion
# ==============================

# Salary at promotion year (first Full year after Associate)
salary_at_promotion_df = (
    salary_clean_df
    .merge(promotion_year_df, on="id", how="inner")
    .loc[lambda df: df["year"] == df["promotion_year"], ["id", "salary"]]
    .rename(columns={"salary": "salary_at_promotion"})
)

# Last Associate year before promotion
assoc_before_promotion_df = (
    associate_df
    .merge(promotion_year_df, on="id", how="inner")
    .loc[
        (associate_df["rank"] == "Assoc"),
        ["id", "year", "salary", "promotion_year"]
    ]
)

assoc_before_promotion_df = assoc_before_promotion_df[
    assoc_before_promotion_df["year"] < assoc_before_promotion_df["promotion_year"]
]

assoc_before_promotion_df = (
    assoc_before_promotion_df
    .sort_values(["id", "year"])
    .groupby("id", as_index=False)
    .tail(1)
    .rename(columns={
        "year": "associate_year_before_promotion",
        "salary": "salary_before_promotion"
    })
    [["id", "associate_year_before_promotion", "salary_before_promotion"]]
)

promotion_event_df = (
    promoted_faculty_summary_df
    .merge(assoc_before_promotion_df, on="id", how="left")
    .merge(salary_at_promotion_df, on="id", how="left")
)

promotion_event_df["salary_change_at_promotion"] = (
    promotion_event_df["salary_at_promotion"] - promotion_event_df["salary_before_promotion"]
)

print("promotion_event_df shape:", promotion_event_df.shape)
promotion_event_df.head()

promotion_event_df shape: (547, 15)


,id,sex,deg,yrdeg,field,startyr,first_associate_year,ever_promoted,associate_cohort_bin,assoc_year_centered,promotion_year,associate_year_before_promotion,salary_before_promotion,salary_at_promotion,salary_change_at_promotion
0,10,M,PhD,68,Arts,80,80,1,"(75, 80]",-4.300813,93,92.0,6781.0,7180.0,399.0
1,13,M,PhD,68,Prof,69,76,1,"(75, 80]",-8.300813,82,79.0,2540.0,3322.0,782.0
2,14,M,PhD,79,Other,92,92,1,"(90, 95]",7.699187,94,93.0,8770.0,9025.0,255.0
3,16,M,PhD,73,Other,79,82,1,"(80, 85]",-2.300813,89,88.0,4538.0,5290.0,752.0
4,17,M,PhD,72,Other,72,77,1,"(75, 80]",-7.300813,83,NaN,NaN,2994.0,NaN


In [39]:
OUTPUT_FILE_3 = "promotion_event_dataset.csv"
promotion_event_df.to_csv(OUTPUT_FILE_3, index=False)
print("Wrote:", OUTPUT_FILE_3)

Wrote: promotion_event_dataset.csv
